In [1]:
from gutenbergpy.gutenbergcache import GutenbergCache, GutenbergCacheTypes
import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
import seaborn as sns
import gensim

# load config
with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Helper as H
from Corpus import Corpus
os.chdir(cwd)

from collections import Counter, defaultdict
import itertools

from gensim.models import KeyedVectors
from gensim.test.utils import datapath

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="dark")
import pickle as pkl
from sklearn.decomposition import PCA

/Users/stephentoner/miniconda3/envs/si699proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

work_list = H.load_file('bookid_year.csv', 'csv', config['REPODIR'] + '/' + config['DATADIR'])

In [4]:
work_list.head()
guten_works = tuple(work_list['Gutenberg ID'])

In [5]:
os.chdir(config['REPODIR'])
cache  = GutenbergCache.get_cache()
# Sample Book Ids for Each author
guten_data = [s for s in cache.native_query(
    "SELECT a.authorid, a.name, count(b.id) as book_count from \
    (SELECT * from authors \
    LEFT JOIN book_authors \
    ON id = authorid) as a \
    LEFT JOIN \
    (SELECT * FROM books \
    WHERE gutenbergbookid IN {}) as b \
    ON b.id = a.bookid;".format(guten_works))]

len(guten_data)

1

In [6]:
guten_sample = [s for s in cache.native_query(
    "SELECT DISTINCT b.id, t.id, t.name, t.title, b.gutenbergbookid FROM (SELECT DISTINCT id, gutenbergbookid FROM books  \
    WHERE gutenbergbookid in {} ) as b \
    LEFT JOIN ( \
        SELECT a.name, a.id, a.bookid, titles.name as title FROM \
            (SELECT * from authors \
             LEFT JOIN book_authors \
             ON id = authorid) as a \
        LEFT JOIN titles \
        ON a.bookid = titles.bookid) AS t \
    ON t.bookid = b.id;".format(guten_works))]

In [7]:
df = pd.DataFrame(guten_sample).drop_duplicates(subset=[0, 4])

In [8]:
data_dict = df.rename(columns = {0:'bid', 1:'aid', 2:'author', 3:'title', 4:'gbid'}).merge(work_list, left_on = 'gbid', right_on='Gutenberg ID').to_dict(orient='record')

/var/folders/b1/vw0sn5w90nnccbfn7bskv0jm0000gn/T/ipykernel_30665/4062553802.py:1: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  data_dict = df.rename(columns = {0:'bid', 1:'aid', 2:'author', 3:'title', 4:'gbid'}).merge(work_list, left_on = 'gbid', right_on='Gutenberg ID').to_dict(orient='record')


In [9]:
from SampleText import fetch_book, trim_book, partition_and_sample

CHUNKS_PER_WORK = 60
CHUNK_LENGTH = 50

# TODO: keep rest of books for training word embeddings?
data = []

for work in data_dict:
    try:
        author_id = work['aid']
        author_name = work['author']
        book_id = work['bid']
        gb_id = work['gbid']
        title = work['title']
        date = work['Date']
        gender = work['Gender']
        penname = work['Penname']
        clean_book, _ = fetch_book(book_id)
        original_len, trimmed_book = trim_book(clean_book)
        excerpts, excerpt_lines = partition_and_sample(
                                            trimmed_book, 
                                            CHUNKS_PER_WORK, 
                                            CHUNK_LENGTH, 
                                            original_len
                                            )
        for idx, passage in enumerate(excerpts):
            data.append({
                "author_id":author_id,
                "author_name":author_name,
                "book_id":book_id,
                "gutenbergbookid":gb_id,
                "title":title,
                "text":passage,
                "text_lines":excerpt_lines[idx],
                "decade":str(date)[:3],
                "gender":gender,
                'penname':penname
            })
    except:
        print("ERROR fetching {}".format(title))
        continue


ERROR fetching Lady Bridget in the Never-Never Land: a story of Australian life
ERROR fetching North and South


In [10]:
with open("data_v4.pkl", "wb") as outfile:
        pkl.dump(data, outfile)

Fetch works for every book

In [137]:
data[0]

In [3]:
corpus = H.load_file('corpus.pkl', 'pkl', config['REPODIR'] + '//' + config['DATADIR'])

In [23]:
data = H.load_file('data.pkl', 'pkl', config['REPODIR'] + '//' + config['DATADIR'])
data_text = [dat['text'] for dat in data]

In [24]:
model = gensim.models.Word2Vec(sentences=data_text, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")